# Assignment 5 - Natural Language Processing

- Student 1 - Luca Modica
- Student 2 - Hugo Alves Henriques E Silva

---

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math

sns.set_style()
%matplotlib inline

## Reading data

In [48]:
from collections import Counter
import re

# Paths to the files
de_file_path = 'dat410_europarl/europarl-v7.de-en.lc.de'
en_de_file_path = 'dat410_europarl/europarl-v7.de-en.lc.en'
fr_file_path = 'dat410_europarl/europarl-v7.fr-en.lc.fr'
en_fr_file_path = 'dat410_europarl/europarl-v7.fr-en.lc.en'
sv_file_path = 'dat410_europarl/europarl-v7.sv-en.lc.sv'
en_sv_file_path = 'dat410_europarl/europarl-v7.sv-en.lc.en'

## Warmup

In [49]:
# Function to read a file and return word frequencies
def get_word_frequencies(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read().lower()  # Ensure all text is lowercase
        words = re.findall(r'\b\w+\b', text)  # Extract words
        word_freq = Counter(words)  # Count word frequencies
    return word_freq


In [50]:
# Get word frequencies for German-English pair
de_word_freq = get_word_frequencies(de_file_path)
en_de_word_freq = get_word_frequencies(en_de_file_path)

# Print the 10 most common words in German and English (German-English pair)
de_common_words = de_word_freq.most_common(10)
en_de_common_words = en_de_word_freq.most_common(10)

# Get word frequencies for French-English pair
fr_word_freq = get_word_frequencies(fr_file_path)
en_fr_word_freq = get_word_frequencies(en_fr_file_path)

# Get word frequencies for Swedish-English pair
sv_word_freq = get_word_frequencies(sv_file_path)
en_sv_word_freq = get_word_frequencies(en_sv_file_path)

# Print the 10 most common words in French, English (French-English pair), Swedish, and English (Swedish-English pair)
fr_common_words = fr_word_freq.most_common(10)
en_fr_common_words = en_fr_word_freq.most_common(10)
sv_common_words = sv_word_freq.most_common(10)
en_sv_common_words = en_sv_word_freq.most_common(10)

print("Most common words in German:", de_common_words)
print("Most common words in English (German-English pair):", en_de_common_words)
print("Most common words in French:", fr_common_words)
print("Most common words in English (French-English pair):", en_fr_common_words)
print("Most common words in Swedish:", sv_common_words)
print("Most common words in English (Swedish-English pair):", en_sv_common_words)



Most common words in German: [('die', 10521), ('der', 9374), ('und', 7028), ('in', 4175), ('zu', 3169), ('den', 2976), ('wir', 2863), ('daß', 2738), ('ich', 2670), ('das', 2669)]
Most common words in English (German-English pair): [('the', 19853), ('of', 9633), ('to', 9069), ('and', 7307), ('in', 6278), ('is', 4478), ('that', 4441), ('a', 4438), ('we', 3372), ('this', 3362)]
Most common words in French: [('apos', 16729), ('de', 14528), ('la', 9746), ('et', 6620), ('l', 6536), ('le', 6177), ('à', 5588), ('les', 5587), ('des', 5232), ('que', 4797)]
Most common words in English (French-English pair): [('the', 19627), ('of', 9534), ('to', 8992), ('and', 7214), ('in', 6197), ('is', 4453), ('that', 4421), ('a', 4388), ('we', 3341), ('this', 3332)]
Most common words in Swedish: [('att', 9181), ('och', 7038), ('i', 5954), ('det', 5687), ('som', 5028), ('för', 4959), ('av', 4013), ('är', 3840), ('en', 3724), ('vi', 3211)]
Most common words in English (Swedish-English pair): [('the', 19327), ('o

In [51]:
# Calculate the total word counts and the counts for 'speaker' and 'zebra' across all English files
total_words = sum(en_de_word_freq.values()) + sum(en_fr_word_freq.values()) + sum(en_sv_word_freq.values())
speaker_count = en_de_word_freq['speaker'] + en_fr_word_freq['speaker'] + en_sv_word_freq['speaker']
zebra_count = en_de_word_freq['zebra'] + en_fr_word_freq['zebra'] + en_sv_word_freq['zebra']

# Calculate probabilities
prob_speaker = speaker_count / total_words
prob_zebra = zebra_count / total_words

print("Total words:", total_words)
print("Speaker count:", speaker_count)
print("Zebra count:", zebra_count)
print("Probability of 'speaker':", prob_speaker)
print("Probability of 'zebra':", prob_zebra)


Total words: 784458
Speaker count: 33
Zebra count: 0
Probability of 'speaker': 4.206726172720528e-05
Probability of 'zebra': 0.0


## Language modeling

In [70]:
from nltk.tokenize import word_tokenize
from nltk.util import bigrams

# Function to tokenize corpus into bigrams with start and end tokens
def create_bigrams(text):
    sentences = text.split('\n')
    bigram_list = []
    for sentence in sentences:
        tokens = ['<START>'] + word_tokenize(sentence)
        bigram_list.extend(list(bigrams(tokens)))
    return bigram_list

# Read the English text files from all three pairs to create a single corpus
corpus_de_en = open(en_de_file_path, 'r', encoding='utf-8').read()
corpus_fr_en = open(en_fr_file_path, 'r', encoding='utf-8').read()
corpus_sv_en = open(en_sv_file_path, 'r', encoding='utf-8').read()

# Combine the corpora
combined_corpus = '\n'.join([corpus_de_en, corpus_fr_en, corpus_sv_en])

# Create bigrams from the combined corpus
bigram_list = create_bigrams(combined_corpus)

# Calculate bigram and unigram counts
unigram_counts = Counter([unigram for bigram in bigram_list for unigram in bigram])
bigram_counts = Counter(bigram_list)

# Function to calculate bigram probabilities using MLE
def calculate_bigram_prob(bigram):
    return bigram_counts[bigram] / unigram_counts[bigram[0]]

# Test the function with an example bigram
example_bigram = ('<START>', 'the')
print("Probability of", example_bigram, ":", calculate_bigram_prob(example_bigram))
example_bigram = ('the', 'zebra')
print("Probability of", example_bigram, ":", calculate_bigram_prob(example_bigram))

Probability of ('<START>', 'the') : 0.11426666666666667
Probability of ('the', 'zebra') : 0.0


In [71]:
def calculate_sentence_prob(sentence):
    sentence_bigram_list = create_bigrams(sentence)
    probability = 1
    for bigram in sentence_bigram_list:
        probability *= calculate_bigram_prob(bigram)
    return probability


print(
    f'Probability of "why are no-smoking areas not enforced ?": {calculate_sentence_prob("why are no-smoking areas not enforced ?")}')
print(
    f'Probability of "the door is green": {calculate_sentence_prob("the door is green")}')
print(
    f'Probability of "we pass": {calculate_sentence_prob("we pass")}')

Probability of "why are no-smoking areas not enforced ?": 7.157743716417319e-18
Probability of "the door is green": 0.0
Probability of "we pass": 2.0531400966183576e-05


When we encounter a word that did not appear in the training texts, this will result in a probability of zero for any bigram containing this word, making the probability of the entire sentence zero. This is a common issue in language modeling known as the zero-probability problem, and it can be handled using techniques like Laplace (add-one) smoothing.

If the sentence is very long, the probability of the sentence will tend to be very small due to the multiplication of probabilities, which can lead to underflow problems in computers. One way to handle this is by working with the log probabilities instead of the raw probabilities.

In [73]:
# Calculate the vocabulary size
vocabulary_size = len(unigram_counts)

# Function to calculate bigram probabilities using Laplace smoothing
def calculate_bigram_log_prob_with_laplace(bigram):
    numerator = bigram_counts[bigram] + 1  # Add one to the count for Laplace smoothing
    denominator = unigram_counts[bigram[0]] + vocabulary_size  # Add vocabulary size for Laplace smoothing
    return math.log(numerator) - math.log(denominator)


#calculate probability of a sentence
def calculate_sentence_prob_improved(sentence):
    tokens = ['<START>'] + word_tokenize(sentence.lower())
    probability = 0
    for i in range(len(tokens) - 1):
        bigram = (tokens[i], tokens[i + 1])
        probability += calculate_bigram_log_prob_with_laplace(bigram)
    return probability


print(
    f'Log probability of "why are no-smoking areas not enforced ?": {calculate_sentence_prob_improved("why are no-smoking areas not enforced ?")}')
print(
    f'Log probability of "the door is open": {calculate_sentence_prob_improved("the door is open")}')
print(
    f'Log probability of "the door is green": {calculate_sentence_prob_improved("the door is green")}')
print(
    f'Log probability of "we pass": {calculate_sentence_prob_improved("we pass")}')


Log probability of "why are no-smoking areas not enforced ?": -57.44415862256026
Log probability of "the door is open": -29.796067530994655
Log probability of "the door is green": -31.741977680049967
Log probability of "we pass": -11.416178508839266


The more negative a log probability is, the less likely the sentence is.

## Translation modeling

In [14]:
def tokenize_corpus(corpus, add_null=False):
    """Tokenize the input corpus (a list of sentences) into a list of lists of tokens.
    Optionally add a NULL token at the beginning of each sentence."""
    tokenized_corpus = [sentence.lower().split() for sentence in corpus]
    if add_null:
        for sentence in tokenized_corpus:
            sentence.insert(0, "null")
    return tokenized_corpus


def initialize_translation_prob(corpus_english, corpus_foreign, add_null=False):
    """Initialize translation probabilities with a lower probability for NULL."""

    word_correspondence = {}

    if add_null:
        for sentence_f in corpus_foreign:
            sentence_f.insert(0, "null")

    for sentence_e, sentence_f in zip(corpus_english, corpus_foreign):
        for word_e in sentence_e:
            if word_e not in word_correspondence:
                word_correspondence[word_e] = []
            for word_f in sentence_f:
                if word_f not in word_correspondence[word_e]:
                    word_correspondence[word_e] += [word_f]


    translation_prob = {}
    null_prob = 0.001

    # print(corpus_foreign)


    for word_e in word_correspondence:
        for word_f in word_correspondence[word_e]:
            if word_f == "null":
               translation_prob[(word_e, word_f)] = null_prob
            else:
                translation_prob[(word_e, word_f)] = (1 - null_prob) / (len(word_correspondence[word_e]) - 1)
            #translation_prob[(word_e, word_f)] = 1 / len(word_correspondence[word_e])

    return translation_prob




print(tokenize_corpus(["The dog runs", "The cat sleeps"]))
print(initialize_translation_prob(tokenize_corpus(["The dog runs", "The cat sleeps", "I am"]), tokenize_corpus(["Le chien court", "Le chat dort", "Je suis"]), add_null=True))

[['the', 'dog', 'runs'], ['the', 'cat', 'sleeps']]
[['null', 'le', 'chien', 'court'], ['null', 'le', 'chat', 'dort'], ['null', 'je', 'suis']]
{('the', 'null'): 0.001, ('the', 'le'): 0.1998, ('the', 'chien'): 0.1998, ('the', 'court'): 0.1998, ('the', 'chat'): 0.1998, ('the', 'dort'): 0.1998, ('dog', 'null'): 0.001, ('dog', 'le'): 0.333, ('dog', 'chien'): 0.333, ('dog', 'court'): 0.333, ('runs', 'null'): 0.001, ('runs', 'le'): 0.333, ('runs', 'chien'): 0.333, ('runs', 'court'): 0.333, ('cat', 'null'): 0.001, ('cat', 'le'): 0.333, ('cat', 'chat'): 0.333, ('cat', 'dort'): 0.333, ('sleeps', 'null'): 0.001, ('sleeps', 'le'): 0.333, ('sleeps', 'chat'): 0.333, ('sleeps', 'dort'): 0.333, ('i', 'null'): 0.001, ('i', 'je'): 0.4995, ('i', 'suis'): 0.4995, ('am', 'null'): 0.001, ('am', 'je'): 0.4995, ('am', 'suis'): 0.4995}


In [15]:
""" def ibm_model_1(corpus_english, corpus_foreign, iterations=10):
    corpus1_tokens = tokenize_corpus(corpus_english)
    corpus2_tokens = tokenize_corpus(corpus_foreign, add_null=True)
    
    # Initialize translation probabilities uniformly
    translation_prob = initialize_translation_prob(corpus1_tokens, corpus2_tokens)

    #print(translation_prob)
    
    for iteration in range(iterations):
        # Initialize counts
        count = {}
        total = {}
        for word1 in translation_prob:
            total[word1[0]] = 0
            for word2 in translation_prob:
                count[(word1[0], word2[1])] = 0
        
        # E-step: Collect counts
        for sentence1, sentence2 in zip(corpus1_tokens, corpus2_tokens):
            # Compute normalization
            #compute alignment probability for each word in sentence1
            for word1 in sentence1:
                total[word1] = sum(translation_prob[(word1, word2)] for word2 in sentence2)
            
            # update pseudo-counts using the normalized alignment probabilities
            for word1 in sentence1:
                for word2 in sentence2:
                    word_translation_prob = translation_prob[(word1, word2)]
                    count[(word1, word2)] += word_translation_prob / total[word1]
                    total[word1] += word_translation_prob / total[word1]   #pseudo-code does not include this part
        
        # M-step: Estimate probabilities
        for word1 in translation_prob:
            for word2 in translation_prob:
                if total[word1[0]] > 0:  # Avoid division by zero
                    translation_prob[(word1[0], word2[1])] = count[(word1[0], word2[1])] / total[word1[0]]
    
    return translation_prob
 """

' def ibm_model_1(corpus_english, corpus_foreign, iterations=10):\n    corpus1_tokens = tokenize_corpus(corpus_english)\n    corpus2_tokens = tokenize_corpus(corpus_foreign, add_null=True)\n    \n    # Initialize translation probabilities uniformly\n    translation_prob = initialize_translation_prob(corpus1_tokens, corpus2_tokens)\n\n    #print(translation_prob)\n    \n    for iteration in range(iterations):\n        # Initialize counts\n        count = {}\n        total = {}\n        for word1 in translation_prob:\n            total[word1[0]] = 0\n            for word2 in translation_prob:\n                count[(word1[0], word2[1])] = 0\n        \n        # E-step: Collect counts\n        for sentence1, sentence2 in zip(corpus1_tokens, corpus2_tokens):\n            # Compute normalization\n            #compute alignment probability for each word in sentence1\n            for word1 in sentence1:\n                total[word1] = sum(translation_prob[(word1, word2)] for word2 in sentence

In [16]:
from collections import defaultdict

def ibm_model_1(corpus_english, corpus_foreign, iterations=10):
    # Assuming tokenize_corpus adds a "null" token to the beginning of each English sentence
    # and splits sentences into lists of words.
    corpus_foreign_tokens = tokenize_corpus(corpus_foreign, add_null=True)  # foreign language corpus
    corpus_english_tokens = tokenize_corpus(corpus_english)  # English corpus, with null word

    # Initialize translation probabilities uniformly
    translation_prob = initialize_translation_prob(corpus_english_tokens, corpus_foreign_tokens)

    for iteration in range(iterations):
        count_ef = defaultdict(float)
        total_f = defaultdict(float)
        
        # E-step: Expectation
        for sentence_e, sentence_f in zip(corpus_english_tokens, corpus_foreign_tokens):
            # For each word in the english sentence
            for word_e in sentence_e:
                # Compute normalization factor for the word2
                s_total_word_f = sum(translation_prob[(word_e, word_f)] for word_f in sentence_f)
                # For each word in the foreign sentence
                for word_f in sentence_f:
                    # Calculate delta, which is the proportion of the alignment probability of the word2 to the word1
                    delta = translation_prob[(word_e, word_f)] / s_total_word_f
                    # Update counts
                    count_ef[(word_e, word_f)] += delta
                    total_f[word_f] += delta
        
        # M-step: Maximization
        for (word_e, word_f), count in count_ef.items():
            translation_prob[(word_e, word_f)] = count / total_f[word_f]


        new_dict = {}
        for key, value in translation_prob.items():
            if key[0] not in new_dict:
                new_dict[key[0]] = value
            else:
                new_dict[key[0]] += value

        for key, value in translation_prob.items():
            translation_prob[key] = value / new_dict[key[0]]


            # house - null = 0.001

    return translation_prob



In [17]:
# Example usage (using dummy data):
corpus1 = ["the house", "the book", "a big house"]
corpus2 = ["das haus", "das buch", "ein großes haus"]  # Assuming German for demonstration

print(initialize_translation_prob(tokenize_corpus(corpus1), tokenize_corpus(corpus2, add_null=True)))

# Estimate translation probabilities
translation_prob = ibm_model_1(corpus1, corpus2, iterations=100)

# Find translations for a specific word (e.g., "house")
translations_for_word = {pair[1]: prob for pair, prob in translation_prob.items() if pair[0] == "house"}
# Sort translations by probability
sorted_translations = sorted(translations_for_word.items(), key=lambda item: item[1], reverse=True)

# Print top N translations
print("Top translations for 'house':")
summm = 0
for foreign_word, prob in sorted_translations[:10]:
    print(f"{foreign_word}: {prob}")
    summm += prob
print(summm)



[['null', 'das', 'haus'], ['null', 'das', 'buch'], ['null', 'ein', 'großes', 'haus']]
{('the', 'null'): 0.001, ('the', 'das'): 0.333, ('the', 'haus'): 0.333, ('the', 'buch'): 0.333, ('house', 'null'): 0.001, ('house', 'das'): 0.24975, ('house', 'haus'): 0.24975, ('house', 'ein'): 0.24975, ('house', 'großes'): 0.24975, ('book', 'null'): 0.001, ('book', 'das'): 0.4995, ('book', 'buch'): 0.4995, ('a', 'null'): 0.001, ('a', 'ein'): 0.333, ('a', 'großes'): 0.333, ('a', 'haus'): 0.333, ('big', 'null'): 0.001, ('big', 'ein'): 0.333, ('big', 'großes'): 0.333, ('big', 'haus'): 0.333}
[['null', 'das', 'haus'], ['null', 'das', 'buch'], ['null', 'ein', 'großes', 'haus']]
Top translations for 'house':
haus: 0.6666666666666206
null: 0.33333333333337944
ein: 1.39806784396942e-18
großes: 1.39806784396942e-18
das: 8.584276825010788e-31
1.0


Need to be very careful with the NULL probability. It needs to be way lower than the other probabilities, otherwise the model will always choose the NULL translation, because NULL will be present in every sentence. Increasing the number of iterations will eventually make null the most probable translation for every word.

In [18]:
#Write code that implements the estimation algorithm for IBM model 1.
# Then print, for either Swedish, German, or French, the 10 words that 
#the English word european is most likely to be translated into, according 
#to your estimate. It can be interesting to look at this list of 10 words and
#see how it changes during the EM iterations.

#reduce the size of corpus_de_en and corpus_sv_en
sv_en = open(en_sv_file_path, 'r', encoding='utf-8').read()
#separate the sentences
sv_en = sv_en.split("\n")
corpus_en = sv_en[:]

sv = open(sv_file_path, 'r', encoding='utf-8').read()
#separate the sentences
sv = sv.split("\n")
corpus_sv = sv[:]

#print(corpus_en)
#print(corpus_sv)

# Estimate translation probabilities
translation_prob = ibm_model_1(corpus_en, corpus_sv, iterations=5)

# Find translations for a specific word (e.g., "european")
translations_for_word = {pair[1]: prob for pair, prob in translation_prob.items() if pair[0] == "european"}
# Sort the top 10 translations by probability

suma = 0

for i in translations_for_word:
    suma = suma + translations_for_word[i]
swedish_translations = sorted(translations_for_word.items(), key=lambda item: item[1], reverse=True)[:10]

[['null', 'jag', 'förklarar', 'europaparlamentets', 'session', 'återupptagen', 'efter', 'avbrottet', 'den', '17', 'december', '.', 'jag', 'vill', 'på', 'nytt', 'önska', 'er', 'ett', 'gott', 'nytt', 'år', 'och', 'jag', 'hoppas', 'att', 'ni', 'haft', 'en', 'trevlig', 'semester', '.'], ['null', 'som', 'ni', 'kunnat', 'konstatera', 'ägde', '&quot;', 'den', 'stora', 'år', '2000-buggen', '&quot;', 'aldrig', 'rum', '.', 'däremot', 'har', 'invånarna', 'i', 'ett', 'antal', 'av', 'våra', 'medlemsländer', 'drabbats', 'av', 'naturkatastrofer', 'som', 'verkligen', 'varit', 'förskräckliga', '.'], ['null', 'ni', 'har', 'begärt', 'en', 'debatt', 'i', 'ämnet', 'under', 'sammanträdesperiodens', 'kommande', 'dagar', '.'], ['null', 'till', 'dess', 'vill', 'jag', 'att', 'vi', ',', 'som', 'ett', 'antal', 'kolleger', 'begärt', ',', 'håller', 'en', 'tyst', 'minut', 'för', 'offren', 'för', 'bl.a.', 'stormarna', 'i', 'de', 'länder', 'i', 'europeiska', 'unionen', 'som', 'drabbats', '.'], ['null', 'jag', 'ber', '

In [19]:
print("Top translations for 'european' in Swedish:")

for foreign_word, prob in swedish_translations:
    print(f"{foreign_word}: {prob}")

print(suma)

Top translations for 'european' in Swedish:
europeisk: 0.031210200710030972
europeiska: 0.028526566806046788
europeiskt: 0.02606960952098506
europaparlamentet: 0.01719412108788692
europaparlamentets: 0.01453065241992983
bitter: 0.01200411242767236
lugnas: 0.011629937748564701
valdeltagandet: 0.009103269515693114
europaval: 0.008833475982950567
sa: 0.008655547697343932
0.9999999999999988


## Decoding